# Exp020: Eliciting target grammar skills
This is an analysis for relationships between grammar skills that may be exploited for scaffolding learners.

In [17]:
from dotenv import load_dotenv
load_dotenv()
import os
os.environ['CACHE_DIR'] = os.environ['FAST_CACHE_DIR'].replace("%SLURM_JOB_ID%", os.getenv('SLURM_JOB_ID')) # speed up model loading

import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from tqdm.notebook import tqdm
import torch
import pickle
from scipy.stats import norm

import sys
sys.path.append(f'../source')
import data
import evaluation
import helpers
import models
import importlib
importlib.reload(helpers)

<module 'helpers' from '/cluster/home/dglandorf/grammarctg/experiments/../source/helpers.py'>

Load dialogs and EGP skills that we are interested in.

In [33]:
dialogs = data.get_dialog_data()
turns = helpers.flatten_list_of_lists([d[0] for d in dialogs])


The attention mask defines which other turns are of interest in between-speaker priming. It has lower triangular pattern of attention to every other diagonal up to a certain lag.

In [69]:
n_turns = [len(d[0]) for d in dialogs]
turn_attention = []
n_resp = 2
idx = 0
for length in n_turns:
    for i in range(0, length):
        attention = []
        for k in range(i, min(length-1, i+(2*n_resp-1)), 2): # only attend to next n response
            attention += [idx+k+1]
        turn_attention.append(attention)
    idx += length

Let's classify the grammar in all turns.

In [39]:
skills = helpers.get_high_conf_classifiers()
classifiers = {nr: models.load_classifier(nr, "corpus_training") for nr in skills}
classified_file = '../data/turn_classification.pkl'

In [43]:
max_turns = int(1e10)
sentences = [(idx, sentence) for idx, turn in tqdm(enumerate(turns[:max_turns]), total=len(turns), desc="Sentence tokenization") for sentence in data.sent_tokenize(turn)]
indices, sents = [s[0] for s in sentences], [s[1] for s in sentences]

Sentence tokenization:   0%|          | 0/710640 [00:00<?, ?it/s]

In [12]:
tokenized_inputs = models.bert_tokenizer(sents, return_tensors='pt', max_length=64, padding='max_length', truncation=True)
dataset = TensorDataset(tokenized_inputs['input_ids'], tokenized_inputs['attention_mask'])
dataloader = DataLoader(dataset, batch_size=512, shuffle=False)

clf_hits = {nr: [] for nr in skills}
for input_ids, attention_mask in tqdm(dataloader, desc="Grammar classification"):    
    input_ids, attention_mask = input_ids.to(models.device), attention_mask.to(models.device)
    with torch.no_grad():
        encoded_inputs = models.bert_encoder(input_ids, attention_mask) # encoding is the same for all classifiers
        x = torch.cat(encoded_inputs.hidden_states, dim=-1)
        for nr, clf in classifiers.items():
            max_values, _ = clf.forward_bert(x, attention_mask)
            clf_hits[nr] += (max_values>0.5).cpu().tolist()

with open(classified_file, 'wb') as f:
    pickle.dump(clf_hits, f)

Sentence tokenization:   0%|          | 0/710640 [00:00<?, ?it/s]

Grammar classification:   0%|          | 0/2371 [00:00<?, ?it/s]

In [40]:
with open(classified_file, 'rb') as f:
    clf_hits = pickle.load(f)

Transform the classification into one big indicator matrix

In [44]:
hits = np.zeros([sum(n_turns), len(skills)], dtype=bool)
for idx, (nr, clf_hit) in enumerate(clf_hits.items()):
    hit_indices = np.array(indices)[clf_hit]
    hits[hit_indices,idx] = 1

In [45]:
base_rate = hits.sum(axis=0)/min(max_turns, len(turns))

In [46]:
base_rate

array([0.0002941 , 0.01075509, 0.0044284 , 0.00390352, 0.00116937,
       0.00012946, 0.00939013, 0.0122453 , 0.05582433, 0.00396825,
       0.00505882, 0.00543876, 0.00704435, 0.00227541, 0.00202071,
       0.00289035, 0.0016844 , 0.00027159, 0.00093296, 0.00075144,
       0.00412023, 0.00024344, 0.04089835, 0.04483001, 0.01540724,
       0.01069177, 0.027077  , 0.01082123, 0.02725571, 0.0012932 ,
       0.00024204])

Check how often we have found a certain skill

Let's bring them into a format for statistics:

In [48]:
p_val = lambda z: (1 - norm.cdf(z)) # two-sided p-test

In [145]:
p_thres = 1e-3
primed = {}
for idx, nr in enumerate(skills):
    print(nr)
    turns_with_skill = set(np.where(hits[:,idx])[0])

    antecedent_skills = np.vstack([hits[turn_attention[turn_nr],:].any(axis=0) for turn_nr in turns_with_skill])
    other_skills = np.vstack([hits[turn_attention[turn_nr],:].any(axis=0) for turn_nr in range(len(turns)) if turn_nr not in turns_with_skill])

    # null hypothesis: p1(chance of target skill after prime)=p2(chance of not target skill after prime)
    # in this case: p(base rate of target skill)
    p = base_rate[idx]
    x1 = antecedent_skills.sum(axis=0)
    n1 = len(antecedent_skills)
    p1 = x1 / n1
    x2 = other_skills.sum(axis=0)
    n2 = len(other_skills)
    p2 = x2 / n2
    
    SE = np.sqrt(p * (1 - p) * (1/n1 + 1/n2))
    z = (p1 - p2) / SE
    p = p_val(z)

    indices = np.where(((p1-p2)>0.05))
    print(list(zip(np.array(skills)[indices], (p1-p2)[indices])))
    primed[nr] = list(np.array(skills)[indices])

57
[]
58
[]
59
[(619, 0.054986076273560974)]
69
[]
76
[(619, 0.08029421429804173)]
77
[(619, 0.05789316072273429)]
616
[(618, 0.06825118066620305), (619, 0.050045011648444646), (621, 0.050059317762962956)]
618
[(616, 0.05910301877374047), (618, 0.09046076773051631), (619, 0.09161875884694422)]
619
[(619, 0.10942033133975915)]
621
[(616, 0.13200107371930717), (618, 0.13418241925913368), (619, 0.19478257184029837)]
624
[(619, 0.12330353432522953)]
625
[(619, 0.16144102063730528)]
628
[(619, 0.18890422698684195)]
629
[(619, 0.11872153470521929)]
630
[(619, 0.122533173407217)]
631
[(619, 0.11197503006872904)]
634
[(619, 0.06873479765482438)]
635
[(619, 0.08708255346296576), (625, 0.052078297841107415), (1176, 0.06338116502599461)]
636
[(619, 0.11094346393236357)]
1106
[]
1112
[(619, 0.17899021719310104)]
1116
[(625, 0.12864763029635046), (631, 0.11013729461719923), (1116, 0.09205062393585929), (1175, 0.1312842092115411), (1179, 0.1133801831931909)]
1175
[]
1176
[]
1177
[]
1178
[]
1179
[(11

## Establish causality by simulating an intervention

Now let's use one of them in a random dialog context and simulate the next answer.

In [2]:
model, tokenizer = models.load_generator("meta-llama/Meta-Llama-3-8B-Instruct")#"/cluster/home/dglandorf/models/llama-FT")

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
testset = pd.read_json("../data/task1/test.json")

Sample random contexts and override the constraint

In [154]:
n = 64
nr = 621
cases = testset.sample(n)
cases['constraints']=[[nr]] * n
cases = cases.apply(lambda x: helpers.get_generation_prompt(x, tokenizer.apply_chat_template, system_msg=True), axis=1)

Generate constrained answers

In [155]:
cases['response'] = models.generate(model, tokenizer, list(cases['prompt']), verbose=False, do_sample=False, eos_token_id=[tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")], batch_size=8)

Generate: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:57<00:00,  7.17s/it]


Append the answers to the context that fulfill the constraint and create prompts for an unconstrained answer to simulate the learner

In [156]:
print(len(cases))
cases = cases[(models.probe_model(classifiers[nr], list(cases['response']))[0] > 0.5).numpy()]
print(len(cases))

64
64


In [158]:
cases['context'] = cases.apply(lambda x: x['context'] + [x['response'].replace("A: ", "")], axis=1)
cases = cases.apply(lambda x: helpers.get_generation_prompt(x, tokenizer.apply_chat_template, system_msg=True, unconstrained=True), axis=1)

Generate next response again

In [159]:
cases['response'] = models.generate(model, tokenizer, list(cases['prompt']), verbose=False, do_sample=False, eos_token_id=[tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")], batch_size=8)

Generate: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [01:23<00:00, 10.45s/it]


In [161]:
cases['context'] = cases.apply(lambda x: x['context'] + [x['response'].replace("A: ", "")], axis=1)

In [166]:
cases['context'].sample(1).iloc[0]

['Hmm interesting. World War 2 sounds like a periodic action film. What is the genre of the film?',
 'The genre is drama, and it is a biopic because it details biographical information about Alan Turing.',
 'Wow. Must be quite fun. I have not heard of the director but I have heard of Benedict Cumberbatch. I like his films. What did you enjoy the most about this movie?',
 "Actually, what I enjoyed most was Benedict Cumberbatch's performance. He was excellent!",
 'Would you like to watch another film about a historical figure, like Churchill or Einstein?',
 'Would you like to watch another film about a historical figure, like Churchill or Einstein?',
 "That's a great idea! I'd love to watch a film about Churchill. I've always been fascinated by his leadership during World War 2."]

Test for target structures

In [160]:
for nr in primed[nr]:
    print(nr)
    print((models.probe_model(classifiers[nr], list(cases['response']))[0]>0.5).float().mean())

616
tensor(0.)
618
tensor(0.)
619
tensor(0.4688)


In [112]:
cases['context'].iloc[6]

['Two number 3s, please.',
 'All right. What would you like to drink?',
 'Diet Coke.',
 'Regular or large?',
 'If I had known the party was going to be so boring, I would have brought a book.']

In [111]:
cases['response'].iloc[6]

'If I had studied harder for the exam, I would have passed with flying colors.'